### Mount Drive

In [ ]:
# Mount Drive
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [ ]:
import os
# The path below should point to the directory containing this notebook and 
# the associated utility files
os.chdir('drive/MyDrive/Colab Notebooks/FourthBrain/data/')
!ls

all_contacts.csv			 preprocess_data_clustering.ipynb
df_complete.csv				 Reports
email_field_hist.csv			 sequence_model.ipynb
event_hist.csv				 students_conversion_clean01212021.pkl
hub_spot_export_clean01212021.pkl	 train.csv
hubspot_export.csv			 url_preprocess.ipynb
last-registered-event-field-history.csv  urls_encoded.csv
pageview_hist.csv			 X4lstm.csv


### Load Data

In [ ]:
# Import libraries 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from urllib.parse import urlparse

In [ ]:
# load data 
df = pd.read_csv('df_complete.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Preprocess Data

In [ ]:
# drop samples without pageview history
pageviews = df.drop(list(np.ravel(np.where(
    df['url0'] == 0 
    ))))

# drop useless column
pageviews = pageviews.drop(columns='Unnamed: 0')

In [ ]:
cols2drop = [
 'Became a Lead Date',
 'Became a Subscriber Date',
 'Became an Opportunity Date',
 'First Conversion Date',
 'First marketing email click date',
 'First marketing email open date',
 'First marketing email send date',
 'Last Activity Date',
 'Last marketing email click date',
 'Last marketing email name',
 'Last marketing email open date',
 'Last marketing email send date',
 'First Page Seen',
 'Last Page Seen',
 'Marketing emails opened',
 'Marketing emails delivered',
 'Marketing emails clicked',
 'Opted out of email: Events and Community',
 'Opted out of email: Program Announcements',
 'Opted out of email: The Batch',
 'Unsubscribed from all email',
 'Which of the following online courses have you taken from deeplearning.ai?',
 'Last Registered Event',
 'Last Registered Event Date',
 'Number of event completions',
 'Highest level of completed education',
 'Job Title or Function',
 'IP Country Code',
 'Associated Company ID',]

# drop columns and store in X
X = pageviews.drop(columns=cols2drop)

In [ ]:
# create dictionary of country encodings 
countries = dict(zip(X['IP Country'].unique(), list(range(len(X)))))

# map encodings to X
X['IP Country'] = X['IP Country'].map(countries)

In [ ]:
# create column identifying paid learners where
# (purchase = 1) and (non-purchase = 0)
r,m = np.where(X.iloc[:, 5:9] == 6)
X['purchase'] = 0
X['purchase'].iloc[r] = 1

In [ ]:
# Feature level data 
X['purchase'].value_counts(normalize=True)

0    0.852078
1    0.147922
Name: purchase, dtype: float64

In [ ]:
# save X to csv
X.to_csv('urls_encoded.csv')

In [ ]:
X.iloc[np.where(X['purchase']==1)]

,Contact ID,Average Pageviews,Number of Pageviews,Number of Sessions,IP Country,url0,url1,url2,url3,purchase
82,2354,1,5,3,6,6,7,3,0,1
92,233,3,7,2,0,6,7,1,4,1
233,1239,1,1,1,6,6,0,0,0,1
264,2150,1,6,4,14,6,2,3,6,1
358,1123,5,5,1,18,6,3,6,3,1
...,...,...,...,...,...,...,...,...,...,...
324915,52997151,4,4,1,6,6,7,5,0,1
324919,53015251,2,5,2,67,6,4,2,0,1
324924,53047151,1,4,3,56,6,7,0,0,1
324961,53209701,1,9,5,6,6,7,5,3,1


### Create new columns for purchase year, months and day.

In [ ]:
# load data
pageview_hist = pd.read_csv('pageview_hist.csv', encoding='utf-8').fillna(0)

# collect indices of rows with purchase = 1
c1 = X['Contact ID'].iloc[np.ravel(np.where(X['url0']==6))].astype('int64')
c2 = X['Contact ID'].iloc[np.ravel(np.where(X['url1']==6))].astype('int64')
c3 = X['Contact ID'].iloc[np.ravel(np.where(X['url2']==6))].astype('int64')
c4 = X['Contact ID'].iloc[np.ravel(np.where(X['url3']==6))].astype('int64')

contacts = [c1, c2, c3, c4]

l1 = []
for i in contacts:
  loc = [np.where((pageview_hist['Contact ID'] == i) == True) \
         for i in i.to_list()]

  l1.append(loc)

locations = l1[0] + l1[1] + l1[2] + l1[3]

In [ ]:
df = pageview_hist.iloc[np.ravel(locations).tolist()]

df1 = df['Last Page Seen Change Date'].str.split(expand=True)
df2 = df1[0].str.split(expand=True)
df3 = df2[0].str.split('-', expand=True)

rl = [np.where((X['Contact ID'] == i) == True) for i in\
      pageview_hist['Contact ID'].iloc[df3.index].to_list()]

X['year'] = 0
X['month'] = 0
X['day'] = 0

X['year'].iloc[np.ravel(rl).tolist()] = df3[0]
X['month'].iloc[np.ravel(rl).tolist()] = df3[1]
X['day'].iloc[np.ravel(rl).tolist()] = df3[2]

X.iloc[np.where(X['purchase'] == 1)]

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


,Contact ID,Average Pageviews,Number of Pageviews,Number of Sessions,IP Country,url0,url1,url2,url3,purchase,year,month,day
82,2354,1,5,3,6,6,7,3,0,1,2020,09,30
92,233,3,7,2,0,6,7,1,4,1,2020,09,29
233,1239,1,1,1,6,6,0,0,0,1,2020,09,28
264,2150,1,6,4,14,6,2,3,6,1,2020,06,19
358,1123,5,5,1,18,6,3,6,3,1,2019,12,01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
324915,52997151,4,4,1,6,6,7,5,0,1,2020,11,13
324919,53015251,2,5,2,67,6,4,2,0,1,2020,11,13
324924,53047151,1,4,3,56,6,7,0,0,1,2020,11,13
324961,53209701,1,9,5,6,6,7,5,3,1,2020,11,16


In [ ]:
df = pageview_hist.iloc[np.ravel(locations).tolist()]

df1 = df['Last Page Seen Change Date']

rl = [np.where((X['Contact ID'] == i) == True) for i in\
      pageview_hist['Contact ID'].iloc[df1.index].to_list()]

X['event_time'] = 0

X['event_time'].iloc[np.ravel(rl).tolist()] = df1

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [ ]:
# get list of columns
cols = X.columns.tolist()
# move last element to front position
cols = cols[-1:] + cols[:-1]
# store reorganized and encoded array in train
train = X[cols]
# convert column event_time from object to datetime
#train['event_time'] = train['event_time'].astype('datetime64[ns]')
# convert column year from object to datetime
train['year'] = train['year'].astype('int64')
# convert column month from object to datetime
train['month'] = train['month'].astype('int64')
# convert column day from object to datetime
train['day'] = train['day'].astype('int64')
# check dtypes of train
train.info()

# save file
train.to_csv('train.csv')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 90345 entries, 5 to 325021
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   event_time           90345 non-null  object
 1   Contact ID           90345 non-null  int64 
 2   Average Pageviews    90345 non-null  int64 
 3   Number of Pageviews  90345 non-null  int64 
 4   Number of Sessions   90345 non-null  int64 
 5   IP Country           90345 non-null  int64 
 6   url0                 90345 non-null  int64 
 7   url1                 90345 non-null  int64 
 8   url2                 90345 non-null  int64 
 9   url3                 90345 non-null  int64 
 10  purchase             90345 non-null  int64 
 11  year                 90345 non-null  int64 
 12  month                90345 non-null  int64 
 13  day                  90345 non-null  int64 
dtypes: int64(13), object(1)
memory usage: 10.3+ MB


### Prepare sequence data

In [ ]:
cols = ['event_time','Contact ID','Average Pageviews','Number of Pageviews',
 'Number of Sessions','IP Country','url0','url1','url2','url3','year',
 'month','day','purchase',]

train = train[cols]

non_zeros = train['event_time'].iloc[np.where(train['event_time'] != 0)].astype('datetime64[ns]')
train['event_time'] = non_zeros
train = train.fillna(0)

In [ ]:
# sort events by 'event time'
data = train.iloc[np.where(X['event_time']!=0)]
data = data.sort_values('event_time')

,event_time,Contact ID,Average Pageviews,Number of Pageviews,Number of Sessions,IP Country,url0,url1,url2,url3,year,month,day,purchase
75073,2019-08-14 05:42:00,202025,2,5,2,8,4,3,6,3,2019,8,14,1
96600,2019-08-14 19:28:00,228989,2,2,1,22,6,3,0,0,2019,8,14,1
27356,2019-08-14 19:31:00,99062,7,7,1,6,6,14,4,8,2019,8,14,1
109441,2019-08-14 19:31:00,169509,2,2,1,6,6,3,0,0,2019,8,14,1
52355,2019-08-14 19:32:00,135617,4,4,1,42,3,6,3,3,2019,8,14,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
321658,2020-11-17 18:36:00,53487251,4,4,1,0,6,6,12,0,2020,11,17,1
301191,2020-11-17 19:18:00,50643551,2,4,2,31,6,1,5,0,2020,11,17,1
216384,2020-11-17 19:50:00,22999951,3,6,2,0,6,7,3,3,2020,11,17,1
325011,2020-11-17 19:53:00,53455201,4,9,2,42,6,0,0,0,2020,11,17,1
